# Day Pipeline Test

낮 파이프라인 전체 흐름을 테스트합니다.

```
Assets 로드
  ↓
WorldState 조회
  ↓
LockManager.check_unlocks() — 정보 해금
  ↓
DayController.process() — Tool 실행, turn += 1
  ↓
Delta 적용
  ↓
EndingChecker.check() — 엔딩 체크
  ↓
NarrativeLayer.render_day() 또는 render_ending()
```

## 0. Setup - 프로젝트 루트 탐색, 환경변수 로드

In [ ]:
import sys
import logging
from pathlib import Path
from dotenv import load_dotenv

ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

load_dotenv(ROOT / ".env")
logging.basicConfig(level=logging.INFO, format="%(name)s | %(message)s")

print(f"Project root: {ROOT}")

## 1. 시나리오 로드 + WorldState 준비

In [ ]:
from app.loader import ScenarioLoader
from app.schemas import WorldStatePipeline, NPCState

loader = ScenarioLoader(ROOT / "scenarios")
scenario_id = "coraline"  # 코렐라인 시나리오 사용
assets = loader.load(scenario_id)

print(f"시나리오: {assets.scenario.get('title')}")
print(f"NPC: {assets.get_all_npc_ids()}")
print(f"턴 제한: {assets.get_turn_limit()}")

# 테스트용 WorldStatePipeline (NPCState는 stats dict 사용)
world = WorldStatePipeline(
    turn=2,
    npcs={
        "button_mother": NPCState(npc_id="button_mother", stats={"trust": 5, "fear": 0, "suspicion": 3}),
        "button_father": NPCState(npc_id="button_father", stats={"trust": 4, "fear": 0, "suspicion": 4}),
        "button_daughter": NPCState(npc_id="button_daughter", stats={"trust": 6, "fear": 0, "suspicion": 2}),
    },
    inventory=[],
    vars={
        "humanity": 8,
        "total_suspicion": 3,
        "knife_touched": 0,
    },
)

print(f"\n초기 상태:")
print(f"  턴: {world.turn}")
print(f"  인간성: {world.vars['humanity']}")
print(f"  총 의심도: {world.vars['total_suspicion']}")

## 2. LockManager - 정보 해금 체크

In [ ]:
from app.lock_manager import get_lock_manager

lock_manager = get_lock_manager()
locks_data = assets.extras.get("locks", {})

print("=== LockManager 체크 ===")
lock_result = lock_manager.check_unlocks(world, locks_data)

print(f"새로 해금된 정보: {len(lock_result.newly_unlocked)}개")
for info in lock_result.newly_unlocked:
    print(f"  - [{info.info_id}] {info.info_title}")
    print(f"    {info.description}")
    print(f"    주입 대상: {', '.join(info.allowed_npcs)}")

print(f"\n총 해금된 정보: {len(lock_result.all_unlocked_ids)}개")

## 3. DayController - 낮 턴 실행

In [ ]:
from app.day_controller import get_day_controller

day_controller = get_day_controller()
user_input = "엄마에게 오늘 저녁은 뭐 먹을지 물어본다"

print(f"=== DayController 실행 ===")
print(f"유저 입력: {user_input}\n")

tool_result = day_controller.process(user_input, world, assets)

print(f"\n이벤트 설명:")
for desc in tool_result.event_description:
    print(f"  - {desc}")

print(f"\nstate_delta:")
print(f"  vars: {tool_result.state_delta.get('vars', {})}")
print(f"  npc_stats: {tool_result.state_delta.get('npc_stats', {})}")
print(f"  turn_increment: {tool_result.state_delta.get('turn_increment', 0)}")

## 4. Delta 적용 - WorldState 업데이트

In [ ]:
from app.state import get_world_state_manager

wsm = get_world_state_manager()
user_id = "test_user"

print("=== Delta 적용 ===")
print(f"적용 전 턴: {world.turn}")

# 테스트 world를 store에 먼저 설정 (apply_delta가 이 상태를 베이스로 사용)
wsm.persist(user_id, scenario_id, world)

# Delta 적용
world_after = wsm.apply_delta(user_id, scenario_id, tool_result.state_delta, assets)

print(f"적용 후 턴: {world_after.turn}")
print(f"\n변경된 상태:")
print(f"  인간성: {world.vars.get('humanity')} → {world_after.vars.get('humanity')}")
print(f"  총 의심도: {world.vars.get('total_suspicion')} → {world_after.vars.get('total_suspicion')}")

for npc_id in world.npcs:
    before = world.npcs[npc_id]
    after = world_after.npcs.get(npc_id, before)
    npc_data = assets.get_npc_by_id(npc_id) or {}
    npc_name = npc_data.get("name", npc_id)
    print(f"\n  [{npc_name}]")
    print(f"    trust: {before.stats.get('trust', 0)} → {after.stats.get('trust', 0)}")
    print(f"    suspicion: {before.stats.get('suspicion', 0)} → {after.stats.get('suspicion', 0)}")

## 5. EndingChecker - 엔딩 조건 체크

In [ ]:
from app.ending_checker import check_ending

print("=== EndingChecker ===")
ending_result = check_ending(world_after, assets)

if ending_result.reached:
    print(f"\n🎬 엔딩 도달!")
    print(f"  엔딩 ID: {ending_result.ending.ending_id}")
    print(f"  엔딩 이름: {ending_result.ending.name}")
    print(f"  epilogue_prompt: {ending_result.ending.epilogue_prompt[:100]}...")
    
    ending_info = {
        "ending_id": ending_result.ending.ending_id,
        "name": ending_result.ending.name,
        "epilogue_prompt": ending_result.ending.epilogue_prompt,
    }
else:
    print("엔딩 미도달 - 게임 계속")
    ending_info = None

## 6. NarrativeLayer - 나레이션 생성

In [ ]:
from app.narrative import get_narrative_layer

narrative = get_narrative_layer()

print("=== NarrativeLayer ===")

if ending_info:
    # 엔딩 나레이션
    print("\n[엔딩 나레이션 생성]")
    dialogue = narrative.render_ending(ending_info, world_after, assets)
else:
    # 일반 낮 나레이션 (render_day 대신 render 사용)
    print("\n[낮 나레이션 생성]")
    dialogue = narrative.render(
        world_state=world_after,
        assets=assets,
        event_description=tool_result.event_description,
        state_delta=tool_result.state_delta,
    )

print("\n" + "=" * 60)
print(dialogue)
print("=" * 60)

## 7. 전체 파이프라인 결과 요약

In [ ]:
print("=" * 60)
print("Day Pipeline 테스트 완료")
print("=" * 60)
print(f"\n최종 상태:")
print(f"  턴: {world.turn} → {world_after.turn}")
print(f"  인간성: {world.vars.get('humanity')} → {world_after.vars.get('humanity')}")
print(f"  엔딩 도달: {'예 (' + ending_info['name'] + ')' if ending_info else '아니오'}")
print(f"\n해금된 정보: {len(lock_result.newly_unlocked)}개")
print(f"생성된 나레이션: {len(dialogue)}자")
print(f"\nDebug Info:")
print(f"  - LockManager: {len(lock_result.all_unlocked)}개 해금")
print(f"  - DayController: {len(tool_result.event_description)}개 이벤트")
print(f"  - EndingChecker: {'reached' if ending_result.reached else 'not reached'}")
print(f"  - NarrativeLayer: {'ending' if ending_info else 'day'} mode")

In [ ]:
### 8-2. 인터랙티브 루프 (종료: q / quit)
print("=" * 60)
print(f"  인터랙티브 모드  |  Turn {_world.turn}")
print("  NPC:", ", ".join(_world.npcs.keys()))
print("  종료: q 또는 quit")
print("=" * 60)

while True:
    try:
        text = input(f"\n[Turn {_world.turn}] 입력 > ").strip()
    except (EOFError, KeyboardInterrupt):
        print("\n종료합니다.")
        break

    if not text:
        continue
    if text.lower() in ("q", "quit", "exit"):
        print("종료합니다.")
        break

    result = _controller.process(text, _world, assets)
    log = _controller.decision_log[-1]
    sel = log["tool_selection"]

    print(f"\n  ┌ tool   : {sel['tool_name']}  |  intent = {log['intent']}")
    print(f"  │ args   : {sel['args']}")
    print(f"  │ 사건   : {result.event_description}")
    if result.npc_response:
        print(f"  │ NPC 응답: {result.npc_response}")
    delta = result.state_delta or {}
    if delta.get("npc_stats"):
        print(f"  │ npc_stats: {delta['npc_stats']}")
    if delta.get("vars"):
        print(f"  │ vars delta: {delta['vars']}")
    print(f"  └ Turn {_world.turn} → {_world.turn + 1}")

    _world.turn += 1

In [ ]:
### 8-1. Setup (현재 스키마 기준)
import sys, logging
from pathlib import Path
from dotenv import load_dotenv

ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

load_dotenv(ROOT / ".env")
logging.basicConfig(level=logging.WARNING, format="%(name)s | %(message)s")  # INFO 로그 억제

from app.loader import ScenarioLoader
from app.schemas import WorldStatePipeline, NPCState
from app.day_controller import get_day_controller

loader = ScenarioLoader(ROOT / "scenarios")
scenario_id = "coraline"
assets = loader.load(scenario_id)

_world = WorldStatePipeline(
    turn=1,
    npcs={
        npc_id: NPCState(
            npc_id=npc_id,
            stats=dict(assets.get_npc_by_id(npc_id).get("stats", {})),
        )
        for npc_id in assets.get_all_npc_ids()
    },
    inventory=[],
    vars={
        k: v.get("default", 0)
        for k, v in assets.scenario.get("state_schema", {}).get("vars", {}).items()
    },
)
_controller = get_day_controller()

print(f"시나리오 : {assets.scenario.get('title')}")
print(f"NPC      : {list(_world.npcs.keys())}")
print(f"초기 vars: {_world.vars}")
print("Setup 완료 — 아래 셀을 실행해 인터랙티브 모드 시작")

## 8. 인터랙티브 모드 — 직접 입력하며 파이프라인 확인

기존 셀(1~7)과 독립적으로 동작합니다.  
아래 두 셀을 순서대로 실행하세요.

1. **Setup 셀** — 현재 스키마 기준으로 assets / world 초기화  
2. **Loop 셀** — `input()`으로 직접 입력 → `DayController.process()` 전체 파이프라인 실행  
   종료하려면 `q` 또는 `quit` 입력